In [ ]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

In [ ]:
base_url = 'https://gov.optimism.io'
req = requests.get(base_url)

soup = BeautifulSoup(req.content, 'html.parser')

In [28]:
categories = []
links = []
descriptions = []

sub_categories = []
sub_links = []
associated_categories = []

In [31]:
# Step 1: Extracting main categories, links, and descriptions
rows = soup.select('tbody > tr')

for row in rows:
    category_div = row.find('td', class_='category').find('div', itemprop='itemListElement')

    if category_div:

        heading_tag = category_div.find('h3').find('span', itemprop='name')
        if heading_tag:
            category_text = heading_tag.text.strip()
            categories.append(category_text)

        link_tag = category_div.find('meta', itemprop='url')
        if link_tag:
            category_link = link_tag['content']
            links.append(base_url + category_link)

        description_tag = category_div.find('div', itemprop='description')
        description_text = description_tag.text.strip() if description_tag else 'No description available'
        descriptions.append(description_text)

# Step 2: Visiting each main link to extract subcategories
        sub_response = requests.get(base_url + category_link)
        sub_soup = BeautifulSoup(sub_response.content, 'html.parser')

        subcategories = sub_soup.select('td.main-link a.title')
        if subcategories:
            for subcategory in subcategories:
                subcategory_name = subcategory.text.strip()
                subcategory_link = subcategory['href']

                sub_categories.append(subcategory_name)
                sub_links.append(subcategory_link)
                associated_categories.append(category_text)

        else:
            sub_categories.append('N/A')
            sub_links.append('N/A')
            sub_descriptions.append('No description available')


In [33]:
main_data = {
    'Main Category': categories,
    'Link': links,
    'Description': descriptions
}

sub_data = {
    'Subcategory': sub_categories,
    'Subcategory Link': sub_links,
    'Main Category': associated_categories
}

In [34]:
main_df = pd.DataFrame(main_data)
main_df.to_csv('main_categories.csv', index=False)

sub_df = pd.DataFrame(sub_data)
sub_df.to_csv('subcategories.csv', index=False)

print("Data has been saved to 'main_categories.csv' and 'subcategories.csv'.")

Data has been saved to 'main_categories.csv' and 'subcategories.csv'.


In [35]:
df = pd.read_csv("main_categories.csv")
df.head()

,Main Category,Link,Description
0,Get Started 🌱,https://gov.optimism.io/c/get-started/67,Welcome to the Optimism Collective governance ...
1,Mission Grants 🏹,https://gov.optimism.io/c/mission-grants/69,How to get a grant from the Governance Fund an...
2,Delegates 🏛,https://gov.optimism.io/c/delegates/41,"Info and discussions on voting, delegation, an..."
3,Retro Funding 🔴,https://gov.optimism.io/c/retrofunding/46,Retroactive Public Goods Funding rounds inform...
4,Citizens 👥,https://gov.optimism.io/c/citizens/79,This category is for all things relating to Ci...


In [38]:
df = pd.read_csv("subcategories.csv")
df.head()

,Subcategory,Subcategory Link,Main Category
0,How to Stay up to Date,https://gov.optimism.io/t/how-to-stay-up-to-da...,Get Started 🌱
1,How to Navigate the Forum,https://gov.optimism.io/t/how-to-navigate-the-...,Get Started 🌱
2,About the Optimism Collective,https://gov.optimism.io/t/about-the-optimism-c...,Get Started 🌱
3,Working Constitution of the Optimism Collective,https://gov.optimism.io/t/working-constitution...,Get Started 🌱
4,Governance Season Guides,https://gov.optimism.io/t/governance-season-gu...,Get Started 🌱


In [ ]:
from google.colab import files

files.download("main_categories.csv")
files.download("subcategories.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>